In [1]:
import torch
import numpy as np

In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"
np.random.seed(42)
sz = 100
x = np.random.rand(sz, 1)
y = 1 + 2 * x + 0.1 * np.random.randn(sz, 1)
idx = np.arange(sz)
np.random.shuffle(idx)
sz80 = int(sz * 0.8)
train_idx = idx[:sz80]
val_idx = idx[sz80:]
x_train, y_train = x[train_idx], y[train_idx]
x_val, y_val = x[val_idx], y[val_idx]
x_train_tensor = torch.from_numpy(x_train).float().to(device)
y_train_tensor = torch.from_numpy(y_train).float().to(device)

In [4]:
from torch import optim, nn

class ManualLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.a = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))
        self.b = nn.Parameter(torch.randn(1, requires_grad=True, dtype=torch.float))

    def forward(self, x):
        return self.a + self.b * x

In [12]:
torch.manual_seed(42)

model = ManualLinearRegression().to(device)
print(model.state_dict())

lr = 0.1
n_epochs = 1000
loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    model.train()
    yhat = model(x_train_tensor)
    loss = loss_fn(yhat, y_train_tensor)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

print(model.state_dict())

OrderedDict([('a', tensor([0.3367], device='cuda:0')), ('b', tensor([0.1288], device='cuda:0'))])
OrderedDict([('a', tensor([1.0235], device='cuda:0')), ('b', tensor([1.9690], device='cuda:0'))])


In [13]:
class LayerLinearRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(1, 1)

    def forward(self, x):
        return self.linear(x)

In [14]:
torch.manual_seed(42)

model = LayerLinearRegression().to(device)
print(model.state_dict())

lr = 0.1
n_epochs = 1000
loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.SGD(model.parameters(), lr=lr)

for epoch in range(n_epochs):
    model.train()
    yhat = model(x_train_tensor)
    loss = loss_fn(yhat, y_train_tensor)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

print(model.state_dict())

OrderedDict([('linear.weight', tensor([[0.7645]], device='cuda:0')), ('linear.bias', tensor([0.8300], device='cuda:0'))])
OrderedDict([('linear.weight', tensor([[1.9690]], device='cuda:0')), ('linear.bias', tensor([1.0235], device='cuda:0'))])


In [15]:
model = nn.Sequential(nn.Linear(1, 1)).to(device)

In [16]:
# шаг обучения
def make_train_step(model, loss_fn, optimizer):
    def train_step(x, y):
        model.train()
        yhat = model(x)
        loss = loss_fn(yhat, y)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        return loss.item()

    return train_step

In [19]:
from torch import optim, nn

torch.manual_seed(42)

model = nn.Sequential(nn.Linear(1, 1).to(device))
lr = 0.1
n_epochs = 1000
loss_fn = nn.MSELoss(reduction="mean")
optimizer = optim.SGD(model.parameters(), lr=lr)
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    loss = train_step(x_train_tensor, y_train_tensor) 

print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9690]], device='cuda:0')), ('0.bias', tensor([1.0235], device='cuda:0'))])


In [20]:
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, x_tensor, y_tensor):
        super().__init__()
        self.x = x_tensor
        self.y = y_tensor

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.x)


In [31]:
from torch.utils.data import TensorDataset
x_train_tensor = torch.from_numpy(x_train).float()
y_train_tensor = torch.from_numpy(y_train).float()

train_data = TensorDataset(x_train_tensor, y_train_tensor)
print(train_data[0])

(tensor([0.7713]), tensor([2.4745]))


In [24]:
from torch.utils.data import DataLoader

train_loader = DataLoader(dataset=train_data, batch_size=16, shuffle=True)

In [30]:
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)

print(model.state_dict())

OrderedDict([('0.weight', tensor([[1.9693]], device='cuda:0')), ('0.bias', tensor([1.0236], device='cuda:0'))])


In [35]:
from torch.utils.data.dataset import random_split

x_tensor = torch.from_numpy(x).float()
y_tensor = torch.from_numpy(y).float()

dataset = TensorDataset(x_tensor, y_tensor)
train_dataset, val_dataset = random_split(dataset, [80, 20])
train_loader = DataLoader(dataset=train_dataset, batch_size=16)
val_loader = DataLoader(dataset=val_dataset, batch_size=20)

In [36]:
train_step = make_train_step(model, loss_fn, optimizer)

for epoch in range(n_epochs):
    for x_batch, y_batch in train_loader:
        x_batch = x_batch.to(device)
        y_batch = y_batch.to(device)
        loss = train_step(x_batch, y_batch)
    
    with torch.no_grad():
        for x_val, y_val in val_loader:
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            model.eval()
            yhat = model(x_val)
            val_loss = loss_fn(yhat, y_val)

print(model.state_dict())
print(loss, val_loss)

OrderedDict([('0.weight', tensor([[1.9072]], device='cuda:0')), ('0.bias', tensor([1.0465], device='cuda:0'))])
0.005920608527958393 tensor(0.0112, device='cuda:0')
